In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Standard library imports.
import itertools
import json
import glob

# Related third party imports.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use(['science','notebook'])

# Local application/library specific imports.


def add_cumulative(df):
    for sim_id in df['simulation_id'].unique():
        for sim_round in df['simulation_round'].unique():
            mask = (df['simulation_id'] == sim_id) & (df['simulation_round'] == sim_round)
            df.loc[mask,'acceptable_count_cum'] = df.loc[mask,'acceptable_count'].cumsum()
            df.loc[mask,'active_vSDN_count_cum'] = df.loc[mask,'active_vSDN_count'].cumsum()
            df.loc[mask,'vSDN_count_cum'] = df.loc[mask,'vSDN_count'].cumsum()
            df.loc[mask,'hp_changed_cum'] = df.loc[mask,'hp_changed'].cumsum()
            df.loc[mask,'acceptance_ratio_cum'] = df.loc[mask,'acceptable_count_cum'] / df.loc[mask,'vSDN_count_cum']
    return df

def get_mean(df, sim_timesteps: int = 50):
    df_ = df.drop(labels=['h_list', 'switch_hpair_count'], axis=1)
    return df_.groupby(by=['dynamic_simulation_name', 'timestep'], as_index=False).mean()

def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

# def get_mean(df, sim_timesteps: int = 50):
#     print(df.shape)
#     df_ = df.drop(labels=['h_list', 'switch_hpair_count'], axis=1)
#     print(df_.shape)
#     simulation_size = sim_timesteps * len(df_['simulation_id'].unique())
#     n_simulation = int(len(df_) / simulation_size)
#     df_mean = pd.DataFrame()
#     for i in range(n_simulation):
#         print(df_mean.shape)
#         mask = list(range(i*simulation_size, (i+1)*simulation_size))
#         df_mean = pd.concat([df_mean,df_.loc[mask,:].groupby(by='timestep', as_index=False).mean()], ignore_index=True)
#     return df_mean

## Save json to csv

In [ ]:
network = '25_italy'
simulation_date = '2022-09-02-19-35'
placement_type = 'ilp-hco'

df = pd.read_json(f"../results/{network}/dynamic/{simulation_date}-{network}-{placement_type}.json")
df.latency_factor = df.latency_factor.round(1)

In [ ]:
df.to_csv(f"../results/{network}/2021-11-04-{network}-ilp-hco.csv", index=False)

In [ ]:
df.query(" vSDN_size == 2 ")

## Analyse Data

In [ ]:
df = pd.read_json('../results/37_cost/dynamic/2022-12-04-21-32-26/simulation-group-results.json')
df = add_cumulative(df)

In [ ]:
df.head()

In [ ]:
df.iloc[0]

In [ ]:
df.query("sim_dynamic_type == 'conservative'")['hp_changed']

In [ ]:
sns.relplot(
    data=df, x='timestep', y='active_vSDN_count_cum', hue='sim_dynamic_type', row='simulation_round',
    kind="line", ci=None, height=4, aspect=3
)

In [ ]:
avb = df.query("sim_dynamic_type == 'basic'")['active_vSDN_count_cum'].to_list()
df['active_vSDN_count_cum_baseline'] = avb * 3
df['active_vSDN_count_cum_difference'] = df['active_vSDN_count_cum'] - df['active_vSDN_count_cum_baseline']

In [ ]:
sns.relplot(
    data=df, x='timestep', y='active_vSDN_count_cum_difference', hue='sim_dynamic_type', row='simulation_round',
    kind="line", ci=None, height=4, aspect=3
)

In [ ]:
sns.relplot(data=df, x='timestep', y='active_vSDN_count', hue='sim_dynamic_type', col='simulation_round', kind="line")

In [ ]:
sns.lineplot(data=df, x='timestep', y='hp_changed_cum', hue='sim_dynamic_type')

In [ ]:
sns.relplot(data=df, x='timestep', y='hp_changed_cum', hue='sim_dynamic_type', row='simulation_round', kind="line")

In [ ]:
h_freq = {}

for (sim_round, dynamic_type), sub_df in df.groupby(by=['simulation_round', 'sim_dynamic_type']):
    # print(sim_round, dynamic_type, len(sub_df))
    h_vc = to_1D(sub_df["h_list"]).value_counts()
    h_freq[(sim_round, dynamic_type)] = h_vc.to_dict()

h_freq = pd.DataFrame.from_dict(h_freq, orient='index').fillna(0)
h_freq = h_freq[sorted(h_freq.columns)]
h_freq = h_freq.reset_index()
h_freq = h_freq.rename(columns={'level_0':'simulation_round', 'level_1':'sim_dynamic_type'})

h_freq = pd.melt(h_freq, id_vars=['simulation_round', 'sim_dynamic_type'])

h_freq.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
sns.barplot(data=h_freq, y='value', x='variable', hue='sim_dynamic_type', ax=ax)

In [ ]:
network = '25_italy'
simulation_date = '2021-11-14'

# placement_type = 'ilp-acc'
# dfa = pd.read_json(f"../results/{network}/dynamic/{simulation_date}-{network}-{placement_type}.json")
# dfa.latency_factor = dfa.latency_factor.round(1)
dfh = pd.read_json("../results/25_italy/dynamic/2022-09-02-22-12-25_italy-heu-hco.json")
add_cumulative(dfh)
dfh = get_mean(dfh)
dfh['latency_factor'] = dfh['latency_factor'].round(1)

dfi = pd.read_json("../results/25_italy/dynamic/2022-09-02-22-08-25_italy-ilp-hco.json")
add_cumulative(dfi)
dfi = get_mean(dfi)
dfi['latency_factor'] = dfi['latency_factor'].round(1)

dfa = pd.read_json("../results/25_italy/dynamic/2022-09-02-22-31-25_italy-ilp-acc.json")
add_cumulative(dfa)
dfa = get_mean(dfa)
dfa['latency_factor'] = dfa['latency_factor'].round(1)

# placement_type = 'ilp-opt'
# # dfo = pd.read_json(f"../results/{network}/dynamic/{simulation_date}-{network}-{placement_type}.json")
# # dfo.latency_factor = dfo.latency_factor.round(1)
dfo = pd.read_json("../results/25_italy/dynamic/2022-09-02-22-38-25_italy-ilp-opt.json")
add_cumulative(dfo)
dfo = get_mean(dfo)
dfo['latency_factor'] = dfo['latency_factor'].round(1)


In [ ]:
print(dfa.groupby('latency_factor').mean()[['acceptable_count', 'acceptance_ratio_cum', 'mean_switch_hpair_count']])

In [ ]:
dfa.query("latency_factor == 0.5")[['hp_runtime', 'acceptance_ratio', 'acceptance_ratio_cum', 'active_vSDN_count', 'mean_switch_hpair_count', 'timestep']]

In [ ]:
# Plot cumulative acceptance ratio
fig, ax = plt.subplots()

for color,data in zip(['b', 'y', 'r', 'g'], [[], [], dfa, dfo]):
    if not len(data):
        continue
    
    for L in data['latency_factor'].unique():
        sns.lineplot(x='timestep', y='acceptance_ratio_cum', data=data.query(f"latency_factor == @L"), color=color, alpha=0.7)

ax.legend(handles=[heu, ilpk, ilpa, opt])
ax.grid()

In [ ]:
# Plot active vSDN count
fig, ax = plt.subplots()

for color,data in zip(['b', 'y', 'r', 'g'], [dfh, dfi, dfa, dfo]):
    if not len(data):
        continue
    
    for L in data['latency_factor'].unique():
        sns.lineplot(x='timestep', y='active_vSDN_count', data=data.query(f"latency_factor == @L"),color=color)

ax.legend(handles=[heu, ilpk, ilpa, opt])
ax.grid()

In [ ]:
# Plot active vSDN count and changed H count
fig, ax = plt.subplots()
ax2 = ax.twinx()

for color,data in zip(['r', 'g'], [dfa, dfo]):
    if not len(data):
        continue
    
    for L in data['latency_factor'].unique():
        sns.lineplot(x='timestep', y='active_vSDN_count', data=data.query(f"latency_factor == @L"), color=color, ax=ax)
        sns.lineplot(x='timestep', y='hp_changed', data=data.query("latency_factor == @L"), color=color, ax=ax2, style=True, dashes=[(2,2)])

ax.legend(handles=[ilpa, opt])
ax2.legend([])
ax.grid()

In [ ]:
# Plot mean switch hpair count
fig, ax = plt.subplots()

for color,data in zip(['r', 'g'], [dfa, dfo]):
    if not len(data):
        continue
    
    for L in data['latency_factor'].unique():
        sns.lineplot(x='timestep', y='mean_switch_hpair_count', data=data.query(f"latency_factor == @L"), color=color, ax=ax)

ax.legend(handles=[ilpa, opt])
ax.grid()

## Save data for latex/pgfplots

In [ ]:
filename = "../results/25_italy/dynamic/2021-11-16-09-07-25_italy-ilp-opt.json"
df = pd.read_json(filename)
add_cumulative(df)
df = get_mean(df)
df['latency_factor'] = df['latency_factor'].round(2)

In [ ]:
for simulation_name in df['simulation_name'].unique():
    df.query("`simulation_name` == @simulation_name").to_csv(f"{filename.rstrip('.json')}-{simulation_name}-mean.csv")

In [ ]:
network_name = '37_cost'

for json_file in glob.glob(f'../results/{network_name}/dynamic/*.json'):
    hp_type = json_file[-12:-5]
    df = pd.read_json(json_file)
    add_cumulative(df)
    df = get_mean(df)
    df['latency_factor'] = df['latency_factor'].round(2)

    for simulation_name in df['simulation_name'].unique():
        df.query("`simulation_name` == @simulation_name").to_csv(f"../results/{network_name}/dynamic/{network_name}-{hp_type}-{simulation_name}-mean.csv", index=False)